# ラボ 6: エッセイ ライター

## セットアップとインポート

このセクションでは、より複雑なプロジェクトである AI エッセイ ライターを構築します。まず、環境をセットアップして必要なライブラリをインポートします。Amazon Bedrock と Anthropic の Claude モデルを使用しているため、インポートはそれを反映しています。
ログ記録をセットアップし、Bedrock を構成し、Tavily API キーを取得します。Tavily は、エッセイの情報を収集するために使用する調査ツールです。Tavily API キーが安全に保存されていることを確認してください。

最後に構築する UI の要件は次のとおりです。

1. tavily ai キーを `.env` ファイルに追加します。

2. LangGraph 0.0.53 で実行していることに注意してください。

3. CLI から helper.py を実行し、Web ブラウザーを開いてグラフをステップ実行するか、ノートブックで直接実行します。

In [ ]:
from dotenv import load_dotenv
import os
import sys
import json, re
import pprint
import boto3
from botocore.client import Config
import warnings

warnings.filterwarnings("ignore")
import logging

# import local modules
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Load environment variables from .env file or Secret Manager
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")
tavily_ai_api_key = utils.get_tavily_api("TAVILY_API_KEY", aws_region)

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client
bedrock_rt = boto3.client(
    "bedrock-runtime", region_name=aws_region, config=bedrock_config
)


## エージェントの状態の定義

では、エージェントの状態を定義しましょう。これは、エッセイ ライターに複数のステップがあり、さまざまな情報を追跡する必要があるため、以前のレッスンよりも複雑です。

AgentState という TypedDict を作成します。これには次のものが含まれます:

- task: エッセイのトピックまたは質問
- plan: エッセイの概要
- draft: エッセイの現在のバージョン
- critique: 現在のドラフトに対するフィードバック
- content: Tavily からの調査情報
- revision_number: 行った修正の数
- max_revisions: 行う修正の最大数

これらの要素は、エッセイ作成プロセスを管理し、修正を停止するタイミングを知るのに役立ちます。

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.memory import MemorySaver

from langchain_core.messages import (
    AnyMessage,
    SystemMessage,
    HumanMessage,
    AIMessage,
    ChatMessage,
)

memory = MemorySaver()

In [ ]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

## モデルの設定

Amazon Bedrock 経由で Anthropic の Claude モデルを使用しています。より一貫した出力を得るために、温度を 0 に設定しています。使用しているモデルは claude-3-haiku で、このタスクに適しています。

In [ ]:
from langchain_aws import ChatBedrockConverse

model = ChatBedrockConverse(
    client=bedrock_rt,
    model="anthropic.claude-3-haiku-20240307-v1:0",
    temperature=0,
    max_tokens=None,
)

## プロンプトの定義

当社のエッセイ ライターは、プロセスのさまざまな段階で複数のプロンプトを使用します。

1. **PLAN_PROMPT**: これは、モデルにエッセイのアウトラインを作成するように指示します。

2. **WRITER_PROMPT**: これは、計画と調査に基づいてモデルがエッセイを書くようにガイドします。

3. **REFLECTION_PROMPT**: これは、モデルにエッセイを批評する方法を伝えます。

4. **RESEARCH_PLAN_PROMPT** および **RESEARCH_CRITIQUE_PROMPT**: これらは、調査ステップの検索クエリを生成するのに役立ちます。

各プロンプトは、エッセイ作成プロセス内でモデルが特定のタスクを実行するようにガイドするために慎重に作成されています。

In [ ]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

> 追記　プロンプトの翻訳（PLAN_PROMPT）

```
あなたは、エッセイの高レベルのアウトラインを書くことを任された専門のライターです。\
ユーザーから提供されたトピックについて、そのようなアウトラインを書いてください。エッセイのアウトラインを、関連するメモやセクションの指示とともに提供してください。
```

In [ ]:
WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------
<content>
{content}
</content>"""

> 追記　プロンプトの翻訳（WRITER_PROMPT）

```
あなたは、優れた 5 段落のエッセイを書くことを任されたエッセイ アシスタントです。\
ユーザーのリクエストと最初のアウトラインに合わせて、可能な限り最高のエッセイを作成します。\
ユーザーから批判があった場合は、以前の試みの修正版で応答します。\
必要に応じて、以下のすべての情報を活用してください。
```

In [ ]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

> 追記　プロンプトの翻訳（REFLECTION_PROMPT）

```
あなたはエッセイの提出物を採点する教師です。\
ユーザーの提出物に対する批評と推奨事項を作成します。\
長さ、深さ、スタイルなどのリクエストを含む詳細な推奨事項を提供します。
```

In [ ]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""

> 追記　プロンプトの翻訳（RESEARCH_PLAN_PROMPT）

```
あなたは、次のエッセイを書くときに使用できる情報を提供する責任を負っている研究者です。
関連する情報を収集する検索クエリのリストを生成してください。
最大 3 つのクエリのみ生成してください。
```

In [ ]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""

> 追記　プロンプトの翻訳（RESEARCH_CRITIQUE_PROMPT）

```
あなたは、要求された修正を行う際に使用できる情報を提供する責任を負っている研究者です (以下に概説)。 
関連する情報を収集する検索クエリのリストを生成します。生成するクエリは最大 3 つだけです。
```

### Anthropic モデルによる構造化された出力生成に関する考察:

上記のプロンプトを見てください。

- これらは Anthropic のプロンプト ガイドに従っていますか?
- たとえば REFLECTION_PROMPT の場合、次のような回答構造を要求することで、より一貫性のある出力が得られると思いますか:

```xml
<answer>
  <overall_assessment>
    <strengths>
      <strength_point></strength_point>
      <strength_point></strength_point>
    </strengths>
    <weaknesses>
      <weakness_point></weakness_point>
      <weakness_point></weakness_point>
    </weaknesses>
  </overall_assessment>

...

  <style_and_language>
    <clarity>
      <comment></comment>
      <recommendation></recommendation>
    </clarity>
    <tone>
      <comment></comment>
      <recommendation></recommendation>
    </tone>
    <grammar_and_mechanics>
      <comment></comment>
      <recommendation></recommendation>
    </grammar_and_mechanics>
  </style_and_language>

  <length_assessment>
    <comment></comment>
    <recommendation></recommendation>
  </length_assessment>

  <conclusion>
    <overall_recommendation></overall_recommendation>
    <priority_improvements>
      <improvement></improvement>
      <improvement></improvement>
    </priority_improvements>
  </conclusion>
</answer>
```

- このような構造の利点と欠点は何でしょうか?
- 追加のトークンに価値があるかどうか自問してみてください。詳細でトークンを多用するプロンプトと、より自由形式のプロンプトのどちらに投資すべきでしょうか?

- この出力をどのように解析しますか?

**ヒント:**
LangChain と PyDantic モデルの XMLOutput-Parser を組み合わせることができます。

参考までに、langchain に最近追加された `.with_structured_output(...)` などのメソッドに依存せずに XMLOutput パーサーを使用する方法を以下で説明します。

In [ ]:
from langchain_core.output_parsers.xml import XMLOutputParser

# Create the XMLOutputParser with our Pydantic model
essay_critique_parser = XMLOutputParser()

# Example usage
xml_string = """
<answer>
  <overall_assessment>
    <strengths>
      <strength_point>Clear thesis statement</strength_point>
      <strength_point>Well-structured paragraphs</strength_point>
    </strengths>
    <weaknesses>
      <weakness_point>Lack of detailed examples</weakness_point>
      <weakness_point>Some grammatical errors</weakness_point>
    </weaknesses>
  </overall_assessment>
  <content_evaluation>
    <depth_of_analysis>
      <comment>The analysis lacks depth in some areas.</comment>
      <recommendation>Expand on key points with more detailed explanations.</recommendation>
    </depth_of_analysis>
    <argument_quality>
      <comment>Arguments are logical but could be stronger.</comment>
      <recommendation>Provide more evidence to support your claims.</recommendation>
    </argument_quality>
    <evidence_use>
      <comment>Limited use of supporting evidence.</comment>
      <recommendation>Incorporate more relevant examples and data.</recommendation>
    </evidence_use>
  </content_evaluation>
  <structure_and_organization>
    <comment>The essay has a clear structure but transitions could be improved.</comment>
    <recommendation>Work on smoother transitions between paragraphs.</recommendation>
  </structure_and_organization>
  <style_and_language>
    <clarity>
      <comment>Writing is generally clear but some sentences are convoluted.</comment>
      <recommendation>Simplify complex sentences for better readability.</recommendation>
    </clarity>
    <tone>
      <comment>The tone is appropriate for an academic essay.</comment>
      <recommendation>Maintain this formal tone throughout.</recommendation>
    </tone>
    <grammar_and_mechanics>
      <comment>There are a few grammatical errors and typos.</comment>
      <recommendation>Proofread carefully to eliminate these errors.</recommendation>
    </grammar_and_mechanics>
  </style_and_language>
  <length_assessment>
    <comment>The essay meets the required length.</comment>
    <recommendation>No changes needed in terms of length.</recommendation>
  </length_assessment>
  <conclusion>
    <overall_recommendation>This is a solid essay that could be improved with more depth and better proofreading.</overall_recommendation>
    <priority_improvements>
      <improvement>Deepen analysis with more detailed explanations and examples.</improvement>
      <improvement>Carefully proofread to eliminate grammatical errors and typos.</improvement>
    </priority_improvements>
  </conclusion>
</answer>
"""

# Parse the XML string
parsed_critique = essay_critique_parser.parse(xml_string)
parsed_critique

ただし、必要な答えを得るために `.with_structured_output` を使用することもできます。

In [ ]:
from langchain_aws import ChatBedrock
from langchain_core.pydantic_v1 import BaseModel, Field


class StructuredOutput(BaseModel):
    title: str = Field(..., description="The title of the response")
    content: str = Field(..., description="The main content of the response")
    summary: str = Field(..., description="A brief summary of the content")


llm = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_kwargs={"temperature": 0},
)

structured_llm = llm.with_structured_output(StructuredOutput)

response = structured_llm.invoke("Tell me about artificial intelligence")
response

In [ ]:
print(response.title)

In [ ]:
structured_llm = llm.with_structured_output(
    StructuredOutput, method="xml_mode"
)  # try xml_mode
response = structured_llm.invoke("Tell me about artificial intelligence")
print(f"The title:\t{response.title}\n")
response

In [ ]:
from langchain_core.pydantic_v1 import BaseModel


class Queries(BaseModel):
    queries: List[str]

### Tavily クライアントの設定

私たちはリサーチに Tavily API を使用しています。TavilyClient をインポートし、API キーで初期化します。これにより、Web 検索を実行してエッセイの情報を収集できるようになります。

In [ ]:
from tavily import TavilyClient
import os

tavily = TavilyClient(api_key=tavily_ai_api_key)

## ノード関数の定義

ここで、エッセイ執筆プロセスの個々のコンポーネントを作成します。各関数はグラフ内のノードを表します。

1. plan_node: エッセイのアウトラインを作成します。

2. research_plan_node: 検索クエリを生成し、計画に基づいて情報を取得します。

3. generation_node: エッセイの下書きを作成します。

4. reflection_node: 現在の下書きを批評します。

5. research_critique_node: 批評に基づいて追加の調査を実行します。

6. should_continue: 改訂を続行するか停止するかを決定します。

これらの各関数は Claude モデルと対話し、それに応じてエージェントの状態を更新します。

In [ ]:
def plan_node(state: AgentState):
    messages = [SystemMessage(content=PLAN_PROMPT), HumanMessage(content=state["task"])]
    response = model.invoke(messages)
    return {"plan": response.content}

In [ ]:
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import json


class Queries(BaseModel):
    queries: List[str] = Field(description="List of research queries")


def research_plan_node(state: AgentState):
    # Set up the Pydantic output parser
    parser = PydanticOutputParser(pydantic_object=Queries)

    # Create a prompt template with format instructions
    prompt = PromptTemplate(
        template="Generate research queries based on the given task.\n{format_instructions}\nTask: {task}\n",
        input_variables=["task"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Use the model with the new prompt and parser
    queries_output = model.invoke(prompt.format_prompt(task=state["task"]))

    # Extract the content from the AIMessage
    queries_text = queries_output.content

    # Extract the JSON string from the content
    json_start = queries_text.find("{")
    json_end = queries_text.rfind("}") + 1
    json_str = queries_text[json_start:json_end]

    # Parse the JSON string
    queries_dict = json.loads(json_str)

    # Create a Queries object from the parsed JSON
    parsed_queries = Queries(**queries_dict)

    content = state["content"] or []
    for q in parsed_queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response["results"]:
            content.append(r["content"])
    return {"content": content}

In [ ]:
def generation_node(state: AgentState):
    content = "\n\n".join(state["content"] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}"
    )
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message,
    ]
    response = model.invoke(messages)
    return {
        "draft": response.content,
        "revision_number": state.get("revision_number", 1) + 1,
    }

In [ ]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state["draft"]),
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [ ]:
def research_critique_node(state: AgentState):
    # Set up the Pydantic output parser
    parser = PydanticOutputParser(pydantic_object=Queries)

    # Create a prompt template with format instructions
    prompt = PromptTemplate(
        template="Generate research queries based on the given critique.\n{format_instructions}\nCritique: {critique}\n",
        input_variables=["critique"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Use the model with the new prompt and parser
    queries_output = model.invoke(prompt.format_prompt(critique=state["critique"]))

    # Extract the content from the AIMessage
    queries_text = queries_output.content

    # Extract the JSON string from the content
    json_start = queries_text.find("{")
    json_end = queries_text.rfind("}") + 1
    json_str = queries_text[json_start:json_end]

    # Parse the JSON string
    queries_dict = json.loads(json_str)

    # Create a Queries object from the parsed JSON
    parsed_queries = Queries(**queries_dict)

    content = state["content"] or []
    for q in parsed_queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response["results"]:
            content.append(r["content"])
    return {"content": content}

In [ ]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

## グラフの作成

ノードが定義されたので、グラフを作成できます。LangGraph の StateGraph を使用して、エッセイ作成プロセスのフローを作成します。各ノードをグラフに追加し、プランナーへのエントリ ポイントを設定し、ノード間のエッジを定義します。

ここで重要な部分は、生成ノードの後の条件付きエッジです。これは、should_continue 関数を使用して、反映して修正するか、プロセスを終了するかを決定します。

In [ ]:
builder = StateGraph(AgentState)

In [ ]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [ ]:
builder.set_entry_point("planner")

In [ ]:
builder.add_conditional_edges(
    "generate", should_continue, {END: END, "reflect": "reflect"}
)

In [ ]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [ ]:
graph = builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image

Image(graph.get_graph().draw_png())

## グラフの実行

エッセイ ライターをテストするために、graph.stream メソッドを使用しています。これにより、プロセスの各ステップをその発生時に確認できます。LangChain と LangSmith の違いに関するエッセイを、最大 2 回の修正で作成するように要求しています。

実行中に、各ノードからの出力が表示され、エッセイが計画、調査、執筆、修正の各段階を通じてどのように進化するかがわかります。

In [ ]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream(
    {
        "task": "what is the difference between langchain and langsmith",
        "max_revisions": 2,
        "revision_number": 1,
    },
    thread,
):
    print(s)

## エッセイ ライター インターフェース

最後に、Gradio を使用したシンプルな GUI を使用して、エッセイ ライターとのやり取りを簡単にします。

**重要な注意**: Amazon SageMaker コード エディター内で Gradio を使用するには、アプリを `shared=True` モードで起動する必要があります。これにより、パブリック リンクが作成されます。[セキュリティとファイル アクセス](https://www.gradio.app/guides/sharing-your-app#security-and-file-access) を確認して、セキュリティへの影響を理解してください。

この GUI を使用すると、エッセイのトピックを入力し、エッセイを生成し、プロセスの各ステップの結果を確認できます。また、各ステップの後にプロセスを中断したり、エッセイの現在の状態を確認したり、エッセイを別の方向に導きたい場合はトピックやプランを変更したりすることもできます。

この GUI を使用すると、エッセイ ライターを簡単に試して、入力やプロセスの変更が最終出力にどのように影響するかを確認できます。

これで、AI エッセイ ライター プロジェクトは終了です。これで、幅広いトピックに関するエッセイを調査、執筆、改良できる、複雑で多段階の AI エージェントが完成しました。このプロジェクトでは、さまざまな AI および API サービスを組み合わせて、強力で実用的なアプリケーションを作成する方法を説明します。

In [ ]:
#set magic variables to allow for a reload when changing code without restarting the kernel
%load_ext autoreload
%autoreload 2

import gradio as gr
from helper import ewriter, writer_gui

MultiAgent = ewriter()
app = writer_gui(MultiAgent.graph)
app.launch()

## 演習 - エッセイライターのプロンプトとパーサーを書き直す